# Context Management:

Probably the most common use of context managers is to manage resources, such as connections to files. In many cases, resources need to be handled properly or the underlying operating systems may balk, produce errors, or just cry piteously.




Let's dive right in an use a context manager and talk about what happens under the hood.

*Hat tip to Luciano Ramalho, author of Fluent Python*

In [56]:
with open('war_of_the_worlds.txt') as fin:
    first_line = fin.readline()
    # additional code goes here...
    
len(first_line)    

69


### What just happened?

* `with` calls the `*.__enter__()` method
* in this case, `open`'s `*.__enter__()` method returns the file object
* the label `fin` is bound to the file object
* when control flow exits the `with` block, the `*.__exit__()` method is called

We can print the variable `first_line`

In [57]:
print(first_line)

The Project Gutenberg EBook of The War of the Worlds, by H. G. Wells



We can access the label `fin` and see that `fin` points to an input-output file object.

In [58]:
repr(fin)

"<_io.TextIOWrapper name='war_of_the_worlds.txt' mode='r' encoding='UTF-8'>"

If we look closely, we will see that sure enough, the file object is now closed. 

In [59]:
fin.closed

True

Despite being closed (i.e. not able to access content in the file), we can still access certain attributes about the file object.

In [60]:
fin.encoding

'UTF-8'

Just to confirm that we can't read from the file any longer...

In [61]:
fin.readline()

ValueError: I/O operation on closed file.

How does this compare with opening a file the naive way?

In [62]:
fin = open("war_of_the_worlds.txt")
line_one = fin.readline()
# additional code goes here...
fin.close()

len(line_one)

69

By comparison, this seems like it only saves us one line AND yet basically does the same thing...

OR does it?

Let's cause an `Error` and see what happens...

In [63]:
error_fin_1 = open("war_of_the_worlds.txt")
line_one = error_fin_1.readline()
# additional code goes here...
# blah, blah
# blah, blah
raise Exception  # an ERROR occurs here...
error_fin_1.close()

len(line_one)

Exception: 

If we check... we see that the `Error` prevented our file from **closing**.

In [64]:
error_fin_1.closed

False

Raising a similar `Error` within a context manager on the other hand...

In [65]:
with open('war_of_the_worlds.txt') as error_fin_2:
    first_line = error_fin_2.readline()
    # additional code goes here...
    # code block, blah, blah
    # code block, blah, blah
    raise Exception # an ERROR occurs here...
    
len(first_line) 

Exception: 

...does **NOT** prevent the file from being closed.

In [66]:
error_fin_2.closed

True

### the 'as' clause is optional and will not be used in all context managers
* Some context managers, such as `file objects`... return self 
* Some context managers return `None`


# Looking Glass

Let's take a look at the `Looking Glass class` that Luciano mentions in his book.

Notice, this `class` has an:

* `__enter__()` method
* `__exit__()` method


```python
class LookingGlass:

    def __enter__(self):
        '''Save a reference to sys.stdout.write and overwrite
        it with a new version.'''
        
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        '''Write text, in reverse order to standard out'''
        
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        '''Set everything back to normal'''
        
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please do not divide by zero!')
            return True
```

We have a copy of this `class` in the accompanying file: `mirror.py`

In [67]:
from mirror import LookingGlass

In [68]:
with LookingGlass() as cm_obj:
    print('This is backwards!')
    

!sdrawkcab si sihT


In [69]:
with LookingGlass() as cm_obj:
    print(cm_obj)
    

YKCOWREBBAJ


In [71]:
print(cm_obj)

JABBERWOCKY


In [72]:
print('Back from wonderland')

Back from wonderland


## Using a context manager without a with block

In [73]:
from mirror import LookingGlass
manager = LookingGlass()
manager

In [74]:
monster = manager.__enter__()
print(monster == 'JABBERWOCKY')


eurT


In [75]:
manager.__exit__(None, None, None)

In [76]:
print('hello')

hello


# Where are context managers used?

* `Path` objects in `pathlib`
* `ZipFile` objects in `zipfile`
* `Popen` objects in `subprocess`
* `TarFile` objects in `tarfile`
* `Lock` objects in `threading`




# contextlib

In [81]:
from contextlib import contextmanager

@contextmanager
def open_file(path, mode):
    print('about to open file')
    the_file = open(path, mode)
    yield the_file
    print('closing file')
    the_file.close()

In [81]:
files = []

for x in range(10):
    with open_file('foo.txt', 'w') as infile:
        files.append(infile)

In [81]:
for f in files:
    if not f.closed:
        print('not closed')

about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file
about to open file
closing file


In [82]:
files

[<_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>,
 <_io.TextIOWrapper name='foo.txt' mode='w' encoding='UTF-8'>]